In [1]:
from pathlib import Path
import pandas as pd
from omero_utils.widgets import ROIScatterViz, OMEConnect


In [2]:
data_dir = Path("/home/guillaume/AmuBox/centuribox-guillaume/DataScatterplots/from zong/")


In [3]:

im_name =  "V1a-DMSO-x40-E1-2.tif"
im = Path(im_name)
data_file = data_dir / (im.as_posix()[:-len(im.suffix)]+".csv")
data = pd.read_csv(data_file)

image_id = 18651

In [4]:
sv = ROIScatterViz(image_id, data, host="139.124.86.50")
sv